## 목표
- 핵심: 이 대회는 **voted=2(No)** 를 양성(positive)으로 평가하는 케이스가 많음
  → 그래서 **학습 라벨 y를 (voted==2)** 로 두고, 모델 출력(sigmoid)을 그대로 제출하면 안전

## 팀 공유용 인사이트
1) 제출은 **voted=2 확률**만 넣으면 된다.
2) Q_A는 reverse + Mach(T/V/M, Mach_score)로 구조화하면 신호가 강해진다.
3) Q_E는 원본 20개보다 delay 요약이 안정적이다.
4) 딥러닝은 seed 앙상블로 분산을 줄인다.

## 제출 전 절대 체크
- (11383, 2)
- columns == ['index','voted']
- voted는 0~1 확률


## 0) 설치(필요 시)


In [2]:
# 필요할 때만 실행하세요.
# !pip install --default-timeout=300 tensorflow scikit-learn


## 1) 데이터 로드(공용 경로)


In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 250)
pd.set_option('display.width', 200)

def find_project_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "requirements.txt").exists() or (p / "README.md").exists():
            return p
    raise FileNotFoundError("프로젝트 루트를 찾지 못했습니다. vote-AI 루트에 requirements.txt 또는 README.md가 있는지 확인하세요.")

PROJECT_ROOT = find_project_root(Path.cwd())
DATA_DIR = PROJECT_ROOT / "data" / "raw"

train = pd.read_csv(DATA_DIR / "train.csv")
test  = pd.read_csv(DATA_DIR / "test_x.csv")
sub   = pd.read_csv(DATA_DIR / "sample_submission.csv")

print("train:", train.shape, "test:", test.shape, "sub:", sub.shape)
train.head()


train: (45532, 78) test: (11383, 77) sub: (11383, 2)


,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,30s,2,1,4,Female,1,3,White,Other,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
1,1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,20s,4,2,3,Female,1,1,Asian,Hindu,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1
2,2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30s,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
3,3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20s,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
4,4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,20s,3,1,2,Male,1,2,White,Agnostic,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1


## 2) 스키마 검증


In [4]:
train_cols = [c for c in train.columns if c != 'voted']
test_cols  = list(test.columns)

print('same columns set   :', set(train_cols) == set(test_cols))
print('same column order  :', train_cols == test_cols)


same columns set   : True
same column order  : True


## 3) 타깃 정의(중요)
이 노트북은 **voted=2(No)** 를 양성(1)으로 둡니다.
→ 모델 출력(sigmoid)은 곧 **P(voted=2)** 이고, 그대로 제출하면 안전합니다.


In [5]:
# y=1 means voted==2 (No)
y = (train['voted'] == 2).astype('int32')
X = train.drop(columns=['voted'])

print("raw voted unique:", sorted(train['voted'].unique()))
print("y counts (1=voted==2):")
print(y.value_counts())
print("positive ratio:", float(y.mean()))


raw voted unique: [np.int64(1), np.int64(2)]
y counts (1=voted==2):
voted
1    24898
0    20634
Name: count, dtype: int64
positive ratio: 0.5468242115435298


## 4) 컬럼 그룹 탐지


In [6]:
cols = list(X.columns)
q_like = [c for c in cols if re.match(r"^Q", c)]

QA_cols = sorted([c for c in q_like if re.fullmatch(r"Q[a-z]A", c)])
QE_cols = sorted([c for c in q_like if re.fullmatch(r"Q[a-z]E", c)])

TP_cols = sorted([c for c in cols if re.fullmatch(r"tp\d{2}", c)])
WR_cols = sorted([c for c in cols if re.fullmatch(r"wr_?\d{2}", c)])
WF_cols = sorted([c for c in cols if re.fullmatch(r"wf_?\d{2}", c)])

print("Q_A:", len(QA_cols), "Q_E:", len(QE_cols), "TP:", len(TP_cols), "WR:", len(WR_cols), "WF:", len(WF_cols))


Q_A: 20 Q_E: 20 TP: 10 WR: 13 WF: 3


## 5) Feature Engineering 

### 포함
- Q_A reverse(1등 코드) + Mach(T/V/M) + Mach_score
- delay(sum/root/log/std) + QE drop
- TIPI 5축(Ex/Ag/Con/Es/Op)
- wr/wf 요약
- teenager_ox(10대 여부)

행 삭제 없음(길이 mismatch 방지)


In [7]:
FLIP_PUBLIC = ["QeA", "QfA", "QkA", "QqA", "QrA"]
FLIP_SECRET = ["QaA", "QdA", "QgA", "QiA", "QnA"]

TPS = ['tp01','tp02','tp03','tp04','tp05','tp06','tp07','tp08','tp09','tp10']

# 범주형으로 둘 컬럼(Embedding 대상)
CAT_COLS = ['age_group', 'gender', 'race', 'religion', 'education', 'engnat', 'married', 'urban']

def fe_v3(df: pd.DataFrame, tp_means=None) -> pd.DataFrame:
    df = df.copy()

    # index/hand 제거(존재하면)
    for c in ['index', 'hand']:
        if c in df.columns:
            df = df.drop(columns=[c])

    # ---- Q_A reverse + Mach ----
    for c in QA_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    for c in FLIP_PUBLIC + FLIP_SECRET:
        if c in df.columns:
            df[c] = 6 - df[c]

    df['T'] = df['QcA'] - df['QfA'] + df['QoA'] - df['QrA'] + df['QsA']
    df['V'] = df['QbA'] - df['QeA'] + df['QhA'] + df['QjA'] + df['QmA'] - df['QqA']
    df['M'] = - df['QkA']
    df['Mach_score'] = df[QA_cols].mean(axis=1)

    # ---- Q_E delay 요약 후 QE drop ----
    for c in QE_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce').clip(lower=0)

    delay_sum = df[QE_cols].sum(axis=1)
    df['delay_sum']  = delay_sum
    df['delay_root'] = np.power(delay_sum, 1/10)
    df['delay_log']  = np.log1p(delay_sum)
    df['delay_std']  = df[QE_cols].std(axis=1)
    df = df.drop(columns=QE_cols)

    # ---- wr/wf 요약 ----
    if len(WR_cols) > 0:
        wr = df[WR_cols].apply(pd.to_numeric, errors='coerce')
        df['wr_yes_count'] = wr.sum(axis=1)
        df['wr_yes_ratio'] = wr.mean(axis=1)
    if len(WF_cols) > 0:
        wf = df[WF_cols].apply(pd.to_numeric, errors='coerce')
        df['wf_yes_count'] = wf.sum(axis=1)
        df['wf_yes_ratio'] = wf.mean(axis=1)
    if 'wr_yes_count' in df.columns and 'wf_yes_count' in df.columns:
        df['wr_minus_wf'] = df['wr_yes_count'] - df['wf_yes_count']

    # ---- tp: 0 -> NaN -> 평균대치 ----
    for c in TPS:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df.loc[df[c] == 0, c] = np.nan
    if tp_means is None:
        tp_means = {c: float(df[c].mean()) for c in TPS}
    for c in TPS:
        df[c] = df[c].fillna(tp_means[c])

    # TIPI 5축
    df['Ex']  = df['tp01'] - df['tp06']
    df['Ag']  = df['tp07'] - df['tp02']
    df['Con'] = df['tp03'] - df['tp08']
    df['Es']  = df['tp09'] - df['tp04']
    df['Op']  = df['tp05'] - df['tp10']

    # teenager_ox
    if 'age_group' in df.columns:
        df['teenager_ox'] = (df['age_group'].astype(str) == '10s').astype('int32')

    # 범주형 문자열화(Embedding)
    for c in CAT_COLS:
        if c in df.columns:
            df[c] = df[c].astype(str)

    # 수치형 NaN -> 0 (안전)
    num_cols_now = [c for c in df.columns if c not in CAT_COLS]
    df[num_cols_now] = df[num_cols_now].apply(pd.to_numeric, errors='coerce').fillna(0)

    return df, tp_means

# 미리보기
X_tmp, tp_means_all = fe_v3(X, tp_means=None)
print("FE shape:", X_tmp.shape)
X_tmp.head()


FE shape: (45532, 74)


,QaA,QbA,QcA,QdA,QeA,QfA,QgA,QhA,QiA,QjA,QkA,QlA,QmA,QnA,QoA,QpA,QqA,QrA,QsA,QtA,age_group,education,engnat,familysize,gender,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,T,V,M,Mach_score,delay_sum,delay_root,delay_log,delay_std,wr_yes_count,wr_yes_ratio,wf_yes_count,wf_yes_ratio,wr_minus_wf,Ex,Ag,Con,Es,Op,teenager_ox
0,3.0,4.0,5.0,5.0,4.0,1.0,4.0,4.0,1.0,4.0,2.0,4.0,2.0,1.0,2.0,2.0,4.0,4.0,2.0,1.0,30s,2,1,4,Female,3,White,Other,2.0,2.0,2.000000,1.000000,2.000000,1.0,7.000000,4.0,4.000000,3.0,1,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,4.0,6.0,-2.0,2.95,17874,2.662087,9.791158,323.295058,7,0.538462,0,0.000000,7,1.0,5.000000,-2.000000,3.0000,-1.000000,0
1,1.0,5.0,3.0,1.0,5.0,3.0,5.0,1.0,1.0,3.0,1.0,4.0,1.0,1.0,1.0,5.0,1.0,5.0,4.0,1.0,20s,4,2,3,Female,1,Asian,Hindu,1.0,1.0,2.371058,3.453648,1.000000,2.0,3.000000,4.0,2.823047,4.0,3,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1,0.0,4.0,-1.0,2.60,49565,2.947932,10.811060,1264.031057,8,0.615385,0,0.000000,8,-1.0,2.000000,-1.628942,-0.6306,-3.000000,0
2,2.0,1.0,1.0,2.0,1.0,5.0,2.0,1.0,3.0,2.0,1.0,5.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,30s,3,1,3,Male,2,White,Other,2.0,3.0,1.000000,5.000000,3.000000,4.0,2.000000,6.0,1.000000,3.0,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,-3.0,4.0,-1.0,1.90,29661,2.800390,10.297622,693.931096,10,0.769231,1,0.333333,9,-2.0,-1.000000,-5.000000,-4.0000,0.000000,0
3,3.0,3.0,4.0,3.0,5.0,4.0,2.0,3.0,1.0,4.0,3.0,4.0,4.0,2.0,4.0,4.0,3.0,2.0,4.0,5.0,20s,4,2,0,Female,1,Asian,Hindu,2.0,4.0,1.000000,1.000000,1.000000,3.0,1.000000,3.0,1.000000,3.0,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,6.0,6.0,-3.0,3.35,72714,3.063104,11.194303,5389.724786,5,0.384615,0,0.000000,5,-1.0,-3.000000,-2.000000,0.0000,-2.000000,0
4,5.0,1.0,5.0,4.0,5.0,4.0,5.0,1.0,1.0,5.0,1.0,5.0,4.0,4.0,1.0,4.0,1.0,1.0,1.0,2.0,20s,3,1,2,Male,2,White,Agnostic,1.0,1.0,1.000000,6.000000,2.043918,2.0,2.588953,6.0,2.000000,6.0,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,2.0,5.0,-1.0,3.00,18487,2.671078,9.824877,366.429756,11,0.846154,1,0.333333,10,-1.0,1.588953,-5.000000,-4.0000,-3.956082,0


## 6) 딥러닝 모델 (간단 MLP + BatchNorm + Dropout)
Residual은 빼고, 설명 쉬운 구조로 유지합니다.


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


In [9]:
def build_mlp_from_fold_train(X_fold: pd.DataFrame, cat_cols, num_cols, emb_dim=24, lr=2e-3, dropout=0.3):
    inputs = {}
    enc = []

    # categorical embeddings
    for c in cat_cols:
        inp = keras.Input(shape=(1,), name=c, dtype=tf.string)
        lookup = layers.StringLookup(output_mode='int')
        lookup.adapt(X_fold[c].astype(str).values)  # fold-train only

        vocab = lookup.vocabulary_size()
        dim = min(emb_dim, max(2, int(np.ceil(vocab**0.25) * 2)))

        x = lookup(inp)
        x = layers.Embedding(vocab, dim)(x)
        x = layers.Reshape((dim,))(x)
        inputs[c] = inp
        enc.append(x)

    # numeric
    num_inp = keras.Input(shape=(len(num_cols),), name="num", dtype=tf.float32)
    norm = layers.Normalization()
    norm.adapt(X_fold[num_cols].values.astype('float32'))  # fold-train only
    x_num = norm(num_inp)
    inputs["num"] = num_inp
    enc.append(x_num)

    x = layers.Concatenate()(enc)

    # simple MLP
    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(64, activation='relu')(x)
    out = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=out)
    opt = keras.optimizers.AdamW(learning_rate=lr, weight_decay=5e-2)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[keras.metrics.AUC(name='auc')])
    return model


### 6-1) tf.data 변환


In [10]:
def df_to_dataset(df: pd.DataFrame, y=None, cat_cols=None, num_cols=None, batch_size=256, shuffle=False, seed=42):
    feats = {}
    for c in cat_cols:
        feats[c] = df[c].astype(str).values
    feats["num"] = df[num_cols].values.astype('float32')

    if y is None:
        ds = tf.data.Dataset.from_tensor_slices(feats)
    else:
        ds = tf.data.Dataset.from_tensor_slices((feats, y.values.astype('float32')))

    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(df), 10000), seed=seed, reshuffle_each_iteration=True)

    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)


## 7) CV + seed 앙상블 (OOF 확인)
이 단계의 AUC는 **voted=2를 양성으로 둔 AUC**입니다.


In [11]:
EPOCHS = 30
BATCH  = 256
SEEDS  = [41, 42, 43]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_by_seed = []

for seed in SEEDS:
    oof = np.zeros(len(X), dtype='float32')

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
        X_tr_raw = X.iloc[tr_idx].reset_index(drop=True)
        y_tr = y.iloc[tr_idx].reset_index(drop=True)
        X_va_raw = X.iloc[va_idx].reset_index(drop=True)
        y_va = y.iloc[va_idx].reset_index(drop=True)

        # fold-train tp 평균
        X_tr_fe, tp_means = fe_v3(X_tr_raw, tp_means=None)
        X_va_fe, _ = fe_v3(X_va_raw, tp_means=tp_means)

        cat_cols = [c for c in CAT_COLS if c in X_tr_fe.columns]
        num_cols = [c for c in X_tr_fe.columns if c not in cat_cols]

        # class_weight (y=1이 voted=2)
        pos = float(y_tr.mean()); neg = 1.0 - pos
        class_weight = {0: 1.0, 1: neg / (pos + 1e-9)}

        tf.keras.utils.set_random_seed(seed + fold)
        model = build_mlp_from_fold_train(X_tr_fe, cat_cols, num_cols, lr=2e-3, dropout=0.3)

        tr_ds = df_to_dataset(X_tr_fe, y_tr, cat_cols, num_cols, batch_size=BATCH, shuffle=True, seed=seed+fold)
        va_ds = df_to_dataset(X_va_fe, y_va, cat_cols, num_cols, batch_size=BATCH, shuffle=False)

        callbacks = [
            keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=4, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_auc', mode='max', factor=0.5, patience=2, min_lr=1e-5),
        ]

        model.fit(tr_ds, validation_data=va_ds, epochs=EPOCHS, verbose=0, callbacks=callbacks, class_weight=class_weight)

        pred_va = model.predict(df_to_dataset(X_va_fe, None, cat_cols, num_cols, batch_size=BATCH), verbose=0).reshape(-1)
        pred_va = np.nan_to_num(pred_va, nan=0.5)

        auc = roc_auc_score(y_va, pred_va)
        oof[va_idx] = pred_va
        print(f"[seed {seed}] fold {fold} AUC={auc:.6f}")

    auc_seed = roc_auc_score(y, oof)
    oof_by_seed.append(oof)
    print(f"\n[seed {seed}] OOF AUC: {auc_seed:.6f}")
    print('-'*60)

oof_ens = np.mean(np.vstack(oof_by_seed), axis=0)
ens_auc = roc_auc_score(y, oof_ens)
print("\n Ensemble OOF AUC:", ens_auc)


[seed 41] fold 0 AUC=0.775919
[seed 41] fold 1 AUC=0.768722
[seed 41] fold 2 AUC=0.759046
[seed 41] fold 3 AUC=0.757676
[seed 41] fold 4 AUC=0.765477

[seed 41] OOF AUC: 0.765006
------------------------------------------------------------
[seed 42] fold 0 AUC=0.776895
[seed 42] fold 1 AUC=0.767184
[seed 42] fold 2 AUC=0.759309
[seed 42] fold 3 AUC=0.759070
[seed 42] fold 4 AUC=0.766804

[seed 42] OOF AUC: 0.765482
------------------------------------------------------------
[seed 43] fold 0 AUC=0.776807
[seed 43] fold 1 AUC=0.767346
[seed 43] fold 2 AUC=0.759998
[seed 43] fold 3 AUC=0.757769
[seed 43] fold 4 AUC=0.765405

[seed 43] OOF AUC: 0.765297
------------------------------------------------------------

 Ensemble OOF AUC: 0.766829516418847


## 8) 최종 학습 → 제출 파일 생성
중요: 이 노트북은 y=1을 voted=2로 학습하므로, 예측 확률은 그대로 제출하면 됩니다.


In [12]:
def train_full_predict_one_seed(seed: int):
    X_tr_fe, tp_means = fe_v3(X, tp_means=None)
    T_fe, _ = fe_v3(test, tp_means=tp_means)

    cat_cols = [c for c in CAT_COLS if c in X_tr_fe.columns]
    num_cols = [c for c in X_tr_fe.columns if c not in cat_cols]

    pos = float(y.mean()); neg = 1.0 - pos
    class_weight = {0: 1.0, 1: neg / (pos + 1e-9)}

    tf.keras.utils.set_random_seed(seed)
    model = build_mlp_from_fold_train(X_tr_fe, cat_cols, num_cols, lr=2e-3, dropout=0.3)

    tr_ds = df_to_dataset(X_tr_fe, y, cat_cols, num_cols, batch_size=BATCH, shuffle=True, seed=seed)
    model.fit(tr_ds, epochs=max(10, EPOCHS//2), verbose=0, class_weight=class_weight)

    pred_test = model.predict(df_to_dataset(T_fe, None, cat_cols, num_cols, batch_size=BATCH), verbose=0).reshape(-1)
    return np.nan_to_num(pred_test, nan=0.5)

preds = []
for seed in SEEDS:
    p = train_full_predict_one_seed(seed)
    preds.append(p)
    print(f"seed {seed} done. mean={float(p.mean()):.6f}")

pred_test = np.mean(np.vstack(preds), axis=0).reshape(-1)

# ✅ sample_submission 그대로 (index, voted)
submission = sub.copy()
submission['voted'] = pred_test

print("submission shape:", submission.shape)
print("columns:", submission.columns.tolist())
print(submission['voted'].describe())

assert submission.shape == (11383, 2)
assert submission.columns.tolist() == ['index', 'voted']

out_path = PROJECT_ROOT / "submission_FINAL_v3.csv"
submission.to_csv(out_path, index=False)

print("Saved:", out_path)
submission.head()


seed 41 done. mean=0.522639
seed 42 done. mean=0.528192
seed 43 done. mean=0.506813
submission shape: (11383, 2)
columns: ['index', 'voted']
count    11383.000000
mean         0.519215
std          0.244478
min          0.088556
25%          0.331786
50%          0.431570
75%          0.715396
max          0.998978
Name: voted, dtype: float64
Saved: /Users/admin/Downloads/AI 헬스케어 수업/oz코딩 수업/해커톤 (1)/vote-AI/submission_FINAL_v3.csv


,index,voted
0,0,0.722649
1,1,0.946449
2,2,0.356850
3,3,0.171374
4,4,0.543838
